<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8/%D0%9A%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82DantistCourses/copi_Prototyp_revisor_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Прототип 1 нашей группы Алгоритм

База знаний с Github на 12.07.2024(можно будет дополнять)

Промпт в sistem первый сделали(можно дорабатывать)

Для векторного пространства Faiss предусмотрели блок сохранения в виде ссылки и локально(можно прередавать любому лицу ссылкой)

Ноутбук общий, работали всей командой группы Алгоритма.

In [ ]:
!pip install -q langchain_openai==0.0.2 faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain_community==0.0.11 langchain==0.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# используем секретный ключ

from openai import OpenAI
import os
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# Добавьте сюда ссылку на базу знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1pNchCgZ9mupXA9nORy_L9CxhiW3V21qbf-8e8UEZFJo/edit?usp=sharing') # заполним, когда база знаний будет готова

In [ ]:
# выведем начало документа
data_from_url[:1000]

'\ufeff#ВОЗРАСТ РЕЗЦА (СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ) 28 - 29 сентября 2024\r\n\r\n\r\n  Преподаватель: СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ\r\n  \r\n  Город: Чита\r\n  \r\n  ПРАКТИЧЕСКИЙ КУРС\r\n  28 сентября 2024\r\n  Регистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)\r\n  МАСТЕР-КЛАСС\r\n  29 сентября 2024\r\n  Регистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)\r\n  Место проведения: Будет объявлено дополнительно\r\n  Стоимость семинара - 16 500 рублей. Стоимость семинар+мастер-класс - 42 000 рублей. При внесении аванса 10 000 руб до 28 августа, стоимость 40 000 руб. Оставшуюся часть можно оплатить до даты курса.\r\n  \r\n  Дополнительно можно получить Свидетельство НМО 36 баллов. Учебный план сформирован на портале НМО и утвержден согласно 36 часам. Стоимость 3 000 руб.\r\n  \r\n  ПРОГРАММА :\r\n  Научитесь видеть и уметь воспроизвести зубы разных возрастных групп композитом в прямой технике\r\n  \r\n  Почти н

Предобрабатываем БЗ таким образом, чтобы можно было разметить ее в формате маркдаун разметки

In [ ]:
def text_to_markdown(text):
    # Добавляем заголовок 1 уровня на основе римских чисел (без переноса строки)
    '''# '''

    # и дублируем его строчкой ниже - иначе эта информация перенесется в метаданные, а порой она бывает полезной.
    '''# '''

    def replace_header1(match):
        return f"# {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3}|IV|V)\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст второго уровня
    '''##  '''
    # и дублируем его строчкой ниже
    '''##  '''

    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    return text

In [ ]:
markdown = text_to_markdown(data_from_url)
print(markdown[:10000])

﻿#ВОЗРАСТ РЕЗЦА (СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ) 28 - 29 сентября 2024


  Преподаватель: СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ
  
  Город: Чита
  
  ПРАКТИЧЕСКИЙ КУРС
  28 сентября 2024
  Регистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)
  МАСТЕР-КЛАСС
  29 сентября 2024
  Регистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)
  Место проведения: Будет объявлено дополнительно
  Стоимость семинара - 16 500 рублей. Стоимость семинар+мастер-класс - 42 000 рублей. При внесении аванса 10 000 руб до 28 августа, стоимость 40 000 руб. Оставшуюся часть можно оплатить до даты курса.
  
  Дополнительно можно получить Свидетельство НМО 36 баллов. Учебный план сформирован на портале НМО и утвержден согласно 36 часам. Стоимость 3 000 руб.
  
  ПРОГРАММА :
  Научитесь видеть и уметь воспроизвести зубы разных возрастных групп композитом в прямой технике
  
  Почти на всех курсах в качестве практики выполняют реставрацию резца в макси

In [ ]:
# Инструкция в system

system = load_document_text('https://docs.google.com/document/d/1kKUterIlei1-6d1-xL3sGfq4OVe4JgK_VH-9M69jAMM/edit?usp=sharing') # заполняем параметр ссылкой на составленный промпт. Не забываем открыть доступ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(system[:1000])

﻿Ты известный практикующий врач стоматолог , преподаватель и консультант курсов повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ». Тебе будут задавать вопросы пользователи, а ты будешь отвечать на заданные вопросы. При ответах на вопросы используй базу знаний данную тебе. Ничего от себя не придумывай и отвечай строго по тексту базы знаний. При ответе на первый вопрос всегда вежливо здоровайся и представляйся ( Здравствуйте. Я консультант по курсам повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ».  Чем я могу вам помочь? Я постараюсь максимально подробно рассказать вам о наших курсах и ответить на ваши вопросы.)  Если пользователь назовет свое имя или имя и отчество , то обращайся к нему на вы и по имени или имени отчеству. Будь предельно вежлив и внимателен. Отвечай только на вопросы по представленной тебе базе знаний и старайся мотивировать пользователя к записи на курсы. Если пользователь задает вопрос не относящийся к базе знаний ответь что ты консультант по к

In [ ]:
# делим БЗ на чанки при помощи MarkdownHeaderTextSplitter, так как предварительно мы ее разметили именно таким образом
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2")
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
fragments = markdown_splitter.split_text(markdown)


In [ ]:
# выводим несколько чанков, чтобы убедиться, что все получилось в необходимом нам формате
fragments[0:5]

[Document(page_content='\ufeff#ВОЗРАСТ РЕЗЦА (СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ) 28 - 29 сентября 2024  \nПреподаватель: СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ  \nГород: Чита  \nПРАКТИЧЕСКИЙ КУРС\n28 сентября 2024\nРегистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)\nМАСТЕР-КЛАСС\n29 сентября 2024\nРегистрация с 9-00 до 10-00, курс с 10-00 до 18-00 (с перерывами на кофе-паузы и бизнес-ланч)\nМесто проведения: Будет объявлено дополнительно\nСтоимость семинара - 16 500 рублей. Стоимость семинар+мастер-класс - 42 000 рублей. При внесении аванса 10 000 руб до 28 августа, стоимость 40 000 руб. Оставшуюся часть можно оплатить до даты курса.  \nДополнительно можно получить Свидетельство НМО 36 баллов. Учебный план сформирован на портале НМО и утвержден согласно 36 часам. Стоимость 3 000 руб.  \nПРОГРАММА :\nНаучитесь видеть и уметь воспроизвести зубы разных возрастных групп композитом в прямой технике  \nПочти на всех курсах в качестве практики выполняют реставрацию рез

In [ ]:
len(fragments)

13

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(fragments, embeddings)

Отдельный блок для сохранения базы Faiss

не запускать без надобности.

In [ ]:
# Задаем имя и путь для сохранения файла
folder_path = '/content/drive/MyDrive/'
index_name = "db"

In [ ]:
# Сохраняем db на ваш Google Drive
db.save_local(folder_path=folder_path, index_name=index_name)

In [ ]:
# Путь к сохраненному файлу
saved_file_path = f"{folder_path}{index_name}"

In [ ]:
# Настройка общего доступа и получение ссылки вручную
print(f"Файл сохранен в {saved_file_path}. Перейдите на Google Drive, найдите файл и настройте общий доступ.")

Файл сохранен в /content/drive/MyDrive/db. Перейдите на Google Drive, найдите файл и настройте общий доступ.


In [ ]:
# Вывод ссылки для ручного копирования и настройки общего доступа
drive_link = f"https://drive.google.com/drive/u/0/my-drive"
print(f"Ссылка на ваш Google Drive: {drive_link}")

Ссылка на ваш Google Drive: https://drive.google.com/drive/u/0/my-drive


Конец блока сохранения базы Faiss

In [ ]:
def answer_index(system, topic, search_index, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",             # выбераем модель
        messages=messages,
        temperature=0         # указываем значение
    )
    answer = completion.choices[0].message.content
    return answer

Проверяем работу нейро-консультанта на сформулированных вопросах

In [ ]:
# вопрос пользователя 1
topic= "Здравствуйте, меня зовут Дмитрий."                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Здравствуйте, Дмитрий. Я консультант по курсам повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ». Чем я могу вам помочь? Я постараюсь максимально подробно рассказать вам о наших курсах и ответить на ваши вопросы.'

In [ ]:
# вопрос пользователя 2
topic= "Хотел бы записаться на курс -  ВОЗРАСТ РЕЗЦА (СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ) 28 - 29 сентября 2024, это возможно?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Здравствуйте. Я консультант по курсам повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ». К сожалению, я не имею доступа к информации о доступности мест на конкретные курсы. Я рекомендую вам связаться с организаторами курса напрямую для получения информации о доступности и возможности записи на курс "ВОЗРАСТ РЕЗЦА (СОШНИКОВ АЛЕКСЕЙ СЕРГЕЕВИЧ) 28 - 29 сентября 2024". Они смогут предоставить вам подробную информацию о курсе и процедуре записи.'

In [ ]:
# вопрос пользователя 3
topic= "А какая стоимость у этого курса?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Здравствуйте. Я консультант по курсам повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ». Стоимость данного курса составляет 25 000 рублей. При внесении аванса 10 000 рублей до 5 сентября, стоимость составит 22 000 рублей. Оставшуюся часть можно оплатить ближе к дате курса или на курсе.'

In [ ]:
# вопрос пользователя 4
topic= "По окончании курса я получу какой то документ?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Да, по окончании курса вы получите именной Сертификат ООО «ПРОФЕССИОНАЛ». Этот документ подтверждает ваше участие в курсе повышения квалификации стоматологов и может быть использован для дальнейшего развития вашей карьеры и профессионального роста.'

In [ ]:
# вопрос пользователя 5
topic= "Расскажите о программе этого курса?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Программа курса "Тотальные работы на имплантах" включает в себя обучение основам ортопедического блока работы с имплантами. На курсе будет рассмотрены следующие темы:\n\n- Оттиски с операционной раны: трансфера и трансфер-чек, ложки, изоляция раны, методики и массы.\n- Предварительная регистрация центрического смещения.\n- Методы регистрации центрического смещения при "все на 4/6".\n- Варианты временных конструкций разных типов и их армирование.\n- Лабораторный трансфер-чек и оттиски для постоянной конструкции.\n- Регистрация прикуса и его передача в лабораторию.\n- Постоянные конструкции различных типов.\n- Гигиена в работе с имплантами.\n\nКурс включает как теоретическую, так и практическую части, где каждому участнику предоставляется возможность работать на микроскопе или бинокулярах. По окончании к'

In [ ]:
# вопрос пользователя 6
topic= "Что входит в лекционную часть?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'В лекционной части курса "СПЛИНТ ТЕРАПИЯ. ОККЛЮЗИОННЫЕ И ФУНКЦИОНАЛЬНО ДЕЙСТВУЮЩИЕ ОРТОПЕДИЧЕСКИЕ ПРИСПОСОБЛЕНИЯ. РАЗНОВИДНОСТИ. КЛАССИФИКАЦИИ. ПОКАЗАНИЯ И АЛГОРИТМЫ РАБОТЫ ПРИ РАЗЛИЧНЫХ ВИДАХ ФУНКЦИОНАЛЬНОЙ ПАТОЛОГИИ" преподаватель рассматривает следующие темы:\n\n1. История метода лечения и его общая и частная характеристики.\n2. Существующие классификации и требования к материалам и способам изготовления.\n3. Группа приспособлений, использующих ограниченный участок зубного ряда.\n4. Группа приспособлений, перекрывающих всю окклюзионную поверхность зубного ряда.\n5. Центрическая и эксцентричес'

In [ ]:
# вопрос пользователя 7
topic= "Какая практическая часть у этого курса"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Практическая часть курса "СПЛИНТ ТЕРАПИЯ. ОККЛЮЗИОННЫЕ И ФУНКЦИОНАЛЬНО ДЕЙСТВУЮЩИЕ ОРТОПЕДИЧЕСКИЕ ПРИСПОСОБЛЕНИЯ. РАЗНОВИДНОСТИ. КЛАССИФИКАЦИИ. ПОКАЗАНИЯ И АЛГОРИТМЫ РАБОТЫ ПРИ РАЗЛИЧНЫХ ВИДАХ ФУНКЦИОНАЛЬНОЙ ПАТОЛОГИИ" включает практический интенсив на биомоделях. Участники курса будут отрабатывать хирургические методики под руководством преподавателя. В рамках практической части будут проводиться следующие действия: подготовка к имплантации при пародонтите, регенеративные методики сохранения зубов, закрытие рецессий при пародонтите.'

In [ ]:
# вопрос пользователя 8
topic= "Есть ли такой курс: ФУНКЦИОНАЛЬНОЙ ОРИЕНТИРОВАННЫЙ ПОДХОД В ДИАГНОСТИКЕ И ПЛАНИРОВАНИИ ЛЕЧЕНИЯ ПАТОЛОГИИ ЖЕВАТЕЛЬНОГО АППАРАТА ?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Здравствуйте. Я консультант по курсам повышения квалификации стоматологов фирмы ООО «ПРОФЕССИОНАЛ». Да, у нас есть курс "ФУНКЦИОНАЛЬНОЙ ОРИЕНТИРОВАННЫЙ ПОДХОД В ДИАГНОСТИКЕ И ПЛАНИРОВАНИИ ЛЕЧЕНИЯ ПАТОЛОГИИ ЖЕВАТЕЛЬНОГО АППАРАТА". Этот курс проводит преподаватель ЕРМОШЕНКО РОМАН БОРИСОВИЧ. Следующий семинар состоится 5 октября 2024 года в городе Новосибирск. Стоимость курса составляет 25 000 рублей. При оплате за оба курса до 05.09.2024 стоимость курсов'

In [ ]:
# вопрос пользователя 9
topic=  " Какая стоимость у этого курса?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Стоимость курса "ТОТАЛЬНЫЕ РАБОТЫ НА ИМПЛАНТАХ" с преподавателем ЗЕМАН АЛЕКСЕЙ составляет 25 000 рублей. При внесении аванса 10 000 рублей до 5 сентября, стоимость курса составит 22 000 рублей. Оставшуюся часть можно оплатить ближе к дате курса или на курсе.'

In [ ]:
# вопрос пользователя 10
topic= "Какая лекционная часть этого курса?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Лекционная часть курса "СПЛИНТ ТЕРАПИЯ. ОККЛЮЗИОННЫЕ И ФУНКЦИОНАЛЬНО ДЕЙСТВУЮЩИЕ ОРТОПЕДИЧЕСКИЕ ПРИСПОСОБЛЕНИЯ. РАЗНОВИДНОСТИ. КЛАССИФИКАЦИИ. ПОКАЗАНИЯ И АЛГОРИТМЫ РАБОТЫ ПРИ РАЗЛИЧНЫХ ВИДАХ ФУНКЦИОНАЛЬНОЙ ПАТОЛОГИИ" будет включать следующие темы:\n1. История метода лечения и его общая и частная характеристики.\n2. Группа приспособлений, использующих ограниченный участок зубного ряда.\n3. Группа приспособлений, перекрывающих всю окклюзионную поверхность зубного ряда.\n4. Центрическая и эксцентрическая репозиция.\n5. Щиты, дилататоры, стандартные приспособления, спортивные каппы.\n6. Обсуждение и обмен мнениями.\n\nЭто лишь общая информация о темах, которые будут рассмотрены в лекционной части курса. Более подробную информацию о каждой теме можно получить на сам'

In [ ]:
# вопрос пользователя 11
topic= "Какакая практическая часть у этого курса?"                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'На практической части курса "СПЛИНТ ТЕРАПИЯ. ОККЛЮЗИОННЫЕ И ФУНКЦИОНАЛЬНО ДЕЙСТВУЮЩИЕ ОРТОПЕДИЧЕСКИЕ ПРИСПОСОБЛЕНИЯ. РАЗНОВИДНОСТИ. КЛАССИФИКАЦИИ. ПОКАЗАНИЯ И АЛГОРИТМЫ РАБОТЫ ПРИ РАЗЛИЧНЫХ ВИДАХ ФУНКЦИОНАЛЬНОЙ ПАТОЛОГИИ" вы будете отрабатывать практические навыки на биомоделях. Вы будете выполнять подготовку к имплантации при пародонтите, регенеративные методики сохранения зубов, закрытие рецессий при пародонтите и другие хирургические методики. Практическая часть будет проводиться под руководством преподавателя.'

In [ ]:
# вопрос пользователя 12
topic= "Спасибо."                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Пожалуйста, чем еще я могу вам помочь?'

In [ ]:
# вопрос пользователя 13
topic= "Спасибо. До свидания."                                         # Пишем сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Пожалуйста, рад был помочь! Если у вас возникнут еще вопросы или вам потребуется дополнительная информация о наших курсах, не стесняйтесь обращаться. Желаю вам хорошего дня! До свидания!'